<a href="https://colab.research.google.com/github/jdiazram/pytorch/blob/master/MultiLayer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

from torchvision import datasets, transforms

In [3]:
# Load the data
mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)


Extracting ./datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/raw




Extracting ./datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/raw




Extracting ./datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [19]:
import torch.nn.functional as F
#definition of the class
class mnist_mlp_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28,500)
        self.fc2 = nn.Linear(500,10)
  
    def forward(self,X):
        Z = F.relu(self.fc1(X))
        Y = F.relu(self.fc2(Z))
        #print(Y.shape)
        return Y

In [20]:
## Training
# Instantiate model
batches = iter(train_loader)
my_model = mnist_mlp_model()
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

print("The model: \n{}".format(my_model))

The model: 
mnist_mlp_model(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [21]:
# Iterate through train set minibatchs 
for images, labels in tqdm(train_loader):
    # Zero out the gradients
    optimizer.zero_grad()
    # Forward pass
    x = images.view(-1, 28*28)
    y = my_model.forward(x)
    loss = criterion(y, labels)
    # Backward pass
    loss.backward()
    optimizer.step()

In [22]:
## Testing
correct = 0
total = len(mnist_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images.view(-1, 28*28)
        y = my_model.forward(x)
        
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())
    
print('Test accuracy: {}'.format(correct/total))


Test accuracy: 0.9214000105857849


**Short answer**

How many trainable parameters does your model have? How does this compare to the logisitic regression example?

**A1:**

Trainanle parameters in my model:

784*500 = 392,000

500*10 = 5,000

bias = 510

TOTAL = 397,510

Trainanle parameters in logisitic regression:

784*10 = 7,840

bias = 10

TOTAL = 7,850

**A2:**

The number of parameters of my model is more large than logistic regression, because my model used 1 layer more and 500 neurons more.
